# Pessimistic Neighbourhood Aggregation for States in Reinforcement Learning

Author: Maleakhi Agung Wijaya  
Supervisor: Marcus Hutter, Sultan Javed Majeed  
Date Created: 21/12/2017

In [5]:
import random
import math

## Mountain Car Environment

In [3]:
class MountainCarEnvironment:
    """
    Implementation of Sutton & Barto (1998) Mountain Car Problem environment.
    """
    velocity_boundaries = (-0.07, 0.07)
    position_boundaries = (-1.2, 0.6)  
   
    # Constructor for MountainCarEnvironment
    # Input: agent for the MountainCarEnvironment
    # Output: MountainCarEnvironment object
    def __init__(self, car):
        self.car = car
        self.reset()
        
    # Compute next state (feature)
    # Output: (new velocity, new position)
    def nextState(self, action):
        # Get current state (velocity, position) and the action chosen by the agent
        velocity = self.car.state[0]
        position = self.car.state[1]
        
        # Calculate the new velocity and new position
        velocity += action * 0.001 + math.cos(3*position) * -0.0025
        position += + velocity
        
        new_state = [velocity, position]
        return(new_state)
    
    # Reset to the initial state    
    def reset(self):
        self.car.state[0] = 0.0
        self.car.state[1] = -0.5
        
    # Give reward for each of the chosen action, depending on what the next state that the agent end up in
    # Output: terminal state = 0, non-terminal state = -1
    def calculateReward(self):
        # Get current position of the agent
        position = self.car.state[1]
        
        # Determine the reward given
        if (position >= 0.6):
            return(0)
        else:
            return(-1) 

## Car (Agent)

In [ ]:
class Agent:
    """
    Implementation of agent (car) that will be used in the Mountain Car Environment using the kNN-TD underlying algorithm
    """
    
    # Constructor
    # Input: size of the storage for previous Q values, parameters for how many neighbours which the agent will choose
    def __init__(self, size, k):
        self.state = [0.0, -0.5]
        self.actions = [-1, 0, 1]
        self.q_storage = []
        self.k = k
        self.knn = []
        self.weight = []
        
        # Initialise the storage with random point 
        for i in range(size):
            initial_value = 0
            initial_action = random.randint(-1, 1)
            initial_state = [random.uniform(-0.07, 0.07), random.uniform(-1.2, 0.6)]
            
            # Each data on the array will consist of state, action pair + value
            data = {"state": initial_state, "value": initial_value, "action": initial_action}
            self.q_storage.append(data)
    
    # Select which action to choose, whether left, neutral, or right
    # Output: -1 (left), 0 (neutral), 1 (right)
    def selectAction(self):
        # First call the knn-td algorithm to determine the value of each Q(s,a) pairs
        action_value = kNN_TD(self.state, self.actions)
        
        # Use the epsilon-greedy method to choose value
        epsilon = 0.05
        random_number = random.uniform(0.0, 1.0)
        if (random_number <= epsilon):
            return(random.randint(-1, 1))
        else:
            return(action_value.index(max(action_value)))
      
    # Standardise feature vector given
    # Input: feature vector to be standardised
    # Output: standardised feature vector
    def standardiseState(self, state):
        standardised_state = []
        standardised_velocity = 2 * ((state[0]+0.07) / (0.07+0.07)) - 1
        standardised_position = 2 * ((state[1]+1.2) / (0.6+1.2)) - 1
        standardised_state.append(standardised_velocity)
        standardised_state.append(standardised_position)
        
        return(standardised_state)
    
    # Calculate Euclidean distance between 2 vectors
    # Input: 2 feature vectors
    # Output: distance between them
    def calculateDistance(self, vector1, vector2):
        return(math.sqrt((vector1[0]-vector2[0])**2 + (vector1[1]-vector2[1])**2))
    
    # Apply the kNN algorithm for feature vector and store the data point on the neighbours array
    # Input: feature vector of current state, actions array consisting of all possible actions
    # Output: vector containing the value of taking each action (left, neutral, right)
    def kNN_TD(self, state, actions):
        temp = [] # array consisting of tuple (distance, original index, weight) for each point in the q_storage
        
        # Get the standardised version of state
        standardised_state = standardiseState(state)
        
        # Loop through every element in the storage array and only calculate for particular action
        for action in actions:
            for i in range(len(q_storage)):
                data = self.q_storage[i]
                # Only want to calculate the nearest neighbour state which has the same action
                if (data["action"] == action):
                    vector_2 = data["state"]
                    distance = calculateDistance(standardised_state, vector_2)
                    index = i
                    weight = 1 / (1 + distance**2)
            
                    # Create the tuple and append that to temp
                    temp.append(tuple((distance, index, weight)))
                else:
                    continue
        
            # After we finish looping through all of the point and calculating the standardise distance,
            # Sort the tuple based on the distance and only take k of it and append that to the neighbours array
            # We also need to calculate the total weight to make it into valid probability that we can compute it's expectation
            sorted_temp = sorted(temp, key=lambda x: x[0])
            total_weight = 0
            for i in range(self.k):
                self.weight.append(sorted_temp[i])
                self.knn.append(q_storage[sorted_temp[i][1]])
                total_weight += sorted_temp[i][2]
            
            # Calculate the expected value of the action and append it to the approximate_action array
            expected_value = 0
            for i in range(self.k):
                weight = self.weight[i][2]
                probability = weight / total_weight
                expected_value += probability * self.knn[i]["value"]
                
            approximate_action.append(expected_value)
        
        return(approximate_action)